In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo


$$
\begin{split}
E_x & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x & = \sum_i^N \sum_j^N  \left [ i j | i j \right ] \\
E_x & = \sum_i^N \sum_j^N  \langle  i j | j i  \rangle 
\end{split}
$$

In [2]:
def exchange_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            #print(i,j,mu,nu,kappa,lamda)
                            energy += C[mu,i]*C[nu,j]*C[kappa,i]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


$$
\begin{split}
E_H & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H & = \sum_i^N \sum_j^N  \left [ i i | j j \right ] \\
E_H & = \sum_i^N \sum_j^N  \langle  i j | i j \rangle 
\end{split}
$$

In [3]:
def hartree_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            #print(i,j,mu,nu,kappa,lamda)
                            energy += C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


In [4]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
print(f"The System {mol.atom} has {eri.size} or {eri.shape[0]}^4 elements in 2-electron-intergrals/4-index-integrals matrix with the {mol.basis}-basis")



The System 
    Ne    0.    0.    0.
 has 6561 or 9^4 elements in 2-electron-intergrals/4-index-integrals matrix with the 6-31g-basis


In [5]:
print("Overlap Integrals")
S = mol.intor('int1e_ovlp')
for i in range(0,S.shape[0]):
    for j in range(0,S.shape[1]):
        print(f"S_{i}{j} = {S[i,j]}" )



Overlap Integrals
S_00 = 0.9999999999999998
S_01 = 0.24901801230727033
S_02 = 0.17191310932176385
S_03 = 0.0
S_04 = 0.0
S_05 = 0.0
S_06 = 0.0
S_07 = 0.0
S_08 = 0.0
S_10 = 0.24901801230727028
S_11 = 0.9999999999999998
S_12 = 0.7590309056851063
S_13 = 0.0
S_14 = 0.0
S_15 = 0.0
S_16 = 0.0
S_17 = 0.0
S_18 = 0.0
S_20 = 0.17191310932176382
S_21 = 0.7590309056851063
S_22 = 0.9999999999999999
S_23 = 0.0
S_24 = 0.0
S_25 = 0.0
S_26 = 0.0
S_27 = 0.0
S_28 = 0.0
S_30 = 0.0
S_31 = 0.0
S_32 = 0.0
S_33 = 1.0000000000000002
S_34 = 0.0
S_35 = 0.0
S_36 = 0.4929819049679909
S_37 = 0.0
S_38 = 0.0
S_40 = 0.0
S_41 = 0.0
S_42 = 0.0
S_43 = 0.0
S_44 = 1.0000000000000002
S_45 = 0.0
S_46 = 0.0
S_47 = 0.4929819049679909
S_48 = 0.0
S_50 = 0.0
S_51 = 0.0
S_52 = 0.0
S_53 = 0.0
S_54 = 0.0
S_55 = 1.0000000000000002
S_56 = 0.0
S_57 = 0.0
S_58 = 0.4929819049679909
S_60 = 0.0
S_61 = 0.0
S_62 = 0.0
S_63 = 0.4929819049679909
S_64 = 0.0
S_65 = 0.0
S_66 = 1.0000000000000004
S_67 = 0.0
S_68 = 0.0
S_70 = 0.0
S_71 = 0.0
S_72 = 0

In [6]:
## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

print("*"*24)
print("MO-Coefficent matrix")
for mu,AO in enumerate(mf.mo_coeff):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)

#print("den ?")
#print(np.matmul(mf.mo_coeff.T,mf.mo_coeff))
#print("*"*24)

converged SCF energy = -128.473876870668
************************
MO-Coefficent matrix
Coefficients of mu=0 [ 0.99550796 -0.24482371 -0.         -0.          0.          0.
  0.          0.         -0.12617152]
Coefficients of mu=1 [ 0.02008192  0.54555307 -0.         -0.          0.          0.
  0.          0.          1.46444498]
Coefficients of mu=2 [-0.00378329  0.54850719 -0.         -0.          0.          0.
  0.          0.         -1.43529233]
Coefficients of mu=3 [-0.00000000e+00 -0.00000000e+00 -4.62673422e-02  6.88305239e-01
  2.30714834e-02 -9.19032794e-01 -9.43612023e-17 -3.93171676e-18
 -0.00000000e+00]
Coefficients of mu=4 [-0.00000000e+00  0.00000000e+00  5.95051289e-01  5.15952389e-02
 -3.45960918e-01  1.05466937e-16 -9.18943191e-01 -1.28330766e-02
 -0.00000000e+00]
Coefficients of mu=5 [-0.00000000e+00  0.00000000e+00  3.46713655e-01  3.30024681e-03
  5.96838179e-01  3.86016807e-17  1.28330766e-02 -9.18943191e-01
 -0.00000000e+00]
Coefficients of mu=6 [-0.00000000e

In [7]:
# get j, k and gamma (1RDM) matrix from hf, 
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
C = mf.mo_coeff
gamma = mf.make_rdm1()

# calculate the energy components to see what they are from the matrices the mf object offers you 
print("Energy Components from PySCF Tools:")
print(f"h_0 = {np.trace(np.matmul(h,gamma))}; U = {1/2*np.trace(np.matmul(J,gamma))}; E_x =  {1/4.*np.trace(np.matmul(K,gamma))}")
print(f"h_0 + U + E_x = {np.trace(np.matmul(h,gamma))+1/2.*np.trace(np.matmul(J,gamma))-1/4.*np.trace(np.matmul(K,gamma))}")

# this should also work
print("Energy Components from direct calculations:")
print(f"U = {2*hartree_energy(eri, mf.mo_coeff, mol)} E_x = {exchange_energy(eri, mf.mo_coeff, mol)} ")

Energy Components from PySCF Tools:
h_0 = -182.62284403304207; U = 66.27654297875942; E_x =  12.127575816385713
h_0 + U + E_x = -128.47387687066836
Energy Components from direct calculations:
U = 66.2765429787601 E_x = 12.127575816385821 


In [8]:
print("Text-Book Gamma")
N = mol.nelec[0]
print(N)
MPgamma=np.matmul(C[:,0:N],C[:,0:N].T)*2
for mu,AO in enumerate(MPgamma):
    print(f"Coefficients of mu={mu} {AO}")

print("PySCF Gamma")
for mu,AO in enumerate(gamma):
    print(f"Coefficients of mu={mu} {AO}")





Text-Book Gamma
5
Coefficients of mu=0 [ 2.1019495  -0.22714524 -0.27610773  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=1 [-0.22714524  0.59606288  0.59832762  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=2 [-0.27610773  0.59832762  0.60174891  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=3 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.52874124e-01
  1.39300337e-16 -4.49979610e-17  6.34081778e-01  1.54599082e-16
 -2.63129321e-17]
Coefficients of mu=4 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.39300337e-16
  9.52874124e-01  1.04125011e-15 -4.14229795e-16  6.34081778e-01
  4.93979439e-16]
Coefficients of mu=5 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.49979610e-17
  1.04125011e-15  9.52874124e-01  1.47808604e-17  9.20689304e-17
  6.34081778e-01]
Coefficients of mu=6 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.34081778e-01
 -4.14229795e-16 

In [9]:
# why do i have to multipl with S instead of S^{-1}
occu, naturalC = np.linalg.eigh(np.matmul(S,gamma))
print(f"should be natural occupation numbers, the sum of occupation numbers, i.e. N is {np.sum(occu)}")
print(occu)
print("*"*24)
print("Natural Orbital LC-Coefficent matrix")
for mu,AO in enumerate(naturalC):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)
print(np.matmul(naturalC, naturalC.T))

should be natural occupation numbers, the sum of occupation numbers, i.e. N is 10.000000000000007
[-0.13530418 -0.13530418 -0.13530418 -0.01819234  2.00533614  2.0128562
  2.13530418  2.13530418  2.13530418]
************************
Natural Orbital LC-Coefficent matrix
Coefficients of mu=0 [ 0.          0.          0.         -0.06086668  0.99278677 -0.10329408
  0.          0.          0.        ]
Coefficients of mu=1 [0.         0.         0.         0.70835282 0.1158728  0.69628283
 0.         0.         0.        ]
Coefficients of mu=2 [ 0.          0.          0.         -0.70322936  0.03078822  0.7102961
  0.          0.          0.        ]
Coefficients of mu=3 [-1.05853669e-01 -4.01266394e-01  4.59202238e-01 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.85438468e-01  4.68543697e-17
  3.27476809e-17]
Coefficients of mu=4 [-5.60478676e-01  2.47672801e-01  8.72252877e-02 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.11022302e-16 -5.93014091e-01
 -5.15022208e-01]
Coeffi